In [1]:
import sys,os
# print(os.path.dirname("C:\Users\epena\OneDrive\Escritorio\qml\m_thesis_git"))
sys.path.append('C:\\Users\\epena\\OneDrive\\Escritorio\\qml\\m_thesis_git\\src')

In [2]:
from qiskit import Aer, QuantumCircuit
from qiskit.circuit import Parameter, ParameterVector
# from qiskit.opflow import StateFn, PauliSumOp, AerPauliExpectation, ListOp, Gradient, OperatorStateFn
# from qiskit.utils import QuantumInstance
import matplotlib.pyplot as plt
from qiskit_utils import BinaryObjectiveFunction
from data_utils import circle, plot_data, generate_ds
import numpy as np
# Set a random seed
np.random.seed(42)

ImportError: DLL load failed while importing _imaging: The specified module could not be found.

In [ ]:
# Generate training and test data
num_training = 200
num_test = 2000

X_train, y_train, new_y_train = generate_ds(num_training)
X_test, y_test, new_y_test= generate_ds(num_test)

In [ ]:
# set method to calculcate expected values
# expval = AerPauliExpectation()

# define gradient method
# gradient = Gradient()

# define quantum instances (statevector and sample based)
# qi_sv = QuantumInstance(Aer.get_backend('aer_simulator_statevector'))

In [ ]:
from qiskit_machine_learning.neural_networks import EstimatorQNN
from qiskit.quantum_info import SparsePauliOp

# construct parametrized circuit
inputs = ParameterVector('input', length=4) # 1 extra for label
weights = ParameterVector('weight', length=9)

# 1qubit classifier
qc1 = QuantumCircuit(1)
qc1.u(inputs[0],inputs[1],inputs[2], 0)
qc1.u(weights[0],weights[1],weights[2], 0)
qc1.u(inputs[0],inputs[1],inputs[2], 0)
qc1.u(weights[3],weights[4],weights[5], 0)
qc1.u(inputs[0],inputs[1],inputs[2], 0)
qc1.u(weights[6],weights[7],weights[8], 0)

op1 = SparsePauliOp.from_list([('Z', 1.0)])
op2 = SparsePauliOp.from_list([('Z', -1.0)])

In [ ]:
# construct OpflowQNN with the operator, the input parameters, the weight parameters,
# the expected value, gradient, and quantum instance.
qnn1 = OpflowQNN(op1, inputs, weights, expval, gradient, qi_sv)
qnn2 = OpflowQNN(op2, inputs, weights, expval, gradient, qi_sv)

In [ ]:
from qiskit.algorithms.optimizers import ADAM, L_BFGS_B
from qiskit_machine_learning.utils.loss_functions import L2Loss

losses = []
def callback_fn(avg_loss, weights):
    print("weights: ", weights)
    print("loss: ", avg_loss)
    losses.append(avg_loss)

function = BinaryObjectiveFunction(X_train, new_y_train, qnn1, qnn2, L2Loss(), callback_fn)

In [ ]:
losses = []
# optimizer = ADAM(maxiter=30, lr=0.8)
optimizer = L_BFGS_B(maxiter=50)
# fit_result = optimizer.minimize(
fit_result = optimizer.optimize(
    num_vars=qnn1.num_weights,
    objective_function=function.objective,
    initial_point=algorithm_globals.random.random(qnn1.num_weights),
    gradient_function=function.gradient,
)

In [ ]:
plt.plot(losses)

In [ ]:
ws = [-0.91709316,  1.57980935, -1.61740473,  1.1182353,  -1.56539397,  1.52962569,
 -1.52987669,  0.00201624,  1.47818211]

from test_utils import get_metrics
y_predict, y_tensor, accuracy, [tp, tn, fp, fn] = get_metrics(X_train, qnn1, ws)

In [ ]:
# print('Accuracy:', acc/len(y_tensor))
plt.figure(figsize=(8,8))
for x, y_target, y_ in zip(X_train, y_tensor, y_predict):
    if y_ == 1:
        plt.plot(x[0], x[1], 'bo')
    else:
        plt.plot(x[0], x[1], 'go')
    if y_target != y_:
        plt.scatter(x[0], x[1], s=200, facecolors='none', edgecolors='r', linewidths=1)
plt.show()

In [ ]:
y_predict, y_tensor, accuracy, [tp, tn, fp, fn] = get_metrics(X_test, qnn1, ws)

In [ ]:
# print('Accuracy:', acc/len(y_tensor))
plt.figure(figsize=(8,8))
for x, y_target, y_ in zip(X_test, y_tensor, y_predict):
    if y_ == 1:
        plt.plot(x[0], x[1], 'bo')
    else:
        plt.plot(x[0], x[1], 'go')
    if y_target != y_:
        plt.scatter(x[0], x[1], s=200, facecolors='none', edgecolors='r', linewidths=1)
plt.show()